In [1]:
from engine import trainer
from utils import plot_results, LinearLR, WarmupLR
from models.model import get_model
from torch.utils.data import ConcatDataset 
from dataset import CUBDataset, FGVCAircraft, FOODDataset

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import v2
import torchvision
from torch.utils.data import default_collate
torchvision.disable_beta_transforms_warning()
from engine import val_step


import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd

import yaml
import json
import time
import os


In [2]:
config = yaml.load(open('finetune_config.yaml', 'r'), Loader=yaml.FullLoader)


LEARNING_RATE = [float(i) for i in config["LEARNING_RATE"]]
LEARNING_SCHEDULER = config["LEARNING_SCHEDULER"]
BATCH_SIZE = int(config["BATCH_SIZE"])
NUM_EPOCHS = int(config["NUM_EPOCHS"])

LOSS = config["LOSS"]
LABEL_SMOOTHING = float(config["LABEL_SMOOTHING"])

IMAGE_SIZE = int(config["IMAGE_SIZE"])
INIT_PATH = config["INIT_PATH"]
PRETRAINED = config["PRETRAINED"]
FREEZE = config["FREEZE"]

CUT_UP_MIX = config["CUT_UP_MIX"]
HPC = config["HPC"]

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {DEVICE} device")

Using cuda device


In [12]:
state_dicts = []
val_results = []
MODEL = "ConvTransNeXtBase"
DATASET = 1
# WEIGHT_PATH = "/home/nuren.zhaksylyk/Documents/CV703/CV703_Assignment_1/runs/finetune/2024-01-27_20-46-18/CUB/ConvTransNeXtTiny"
# WEIGHT_PATH = "/home/nuren.zhaksylyk/Documents/CV703/CV703_Assignment_1/runs/finetune/2024-01-28_18-25-52/CUB/ConvTransNeXtBase"
# WEIGHT_PATH = "/home/nuren.zhaksylyk/Documents/CV703/CV703_Assignment_1/runs/finetune/2024-01-28_18-29-01/CUB and FGVC-Aircraft/ConvTransNeXtTiny"
WEIGHT_PATH = "/home/nuren.zhaksylyk/Documents/CV703/CV703_Assignment_1/runs/finetune/2024-01-29_12-51-43/CUB and FGVC-Aircraft/ConvTransNeXtBase"
def START_seed():
    seed = 9
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed) 

In [4]:
START_seed()

dataset_name = ["CUB", "CUB and FGVC-Aircraft", "FoodX"][DATASET]
num_classes = [200, 200 + 100, 251][DATASET]

#run id is date and time of the run

save_root_dir = "./runs/finetune/" + dataset_name + '/' + MODEL
os.makedirs(save_root_dir, exist_ok=True)



#load data
transforms_train = v2.Compose([
    v2.ToImage(),
    v2.RandomResizedCrop((IMAGE_SIZE, IMAGE_SIZE), scale=(0.7, 1.0), antialias=True),

    # v2.AutoAugment(policy=v2.AutoAugmentPolicy.IMAGENET, ),
    v2.RandAugment(num_ops=2, magnitude=10),
    v2.RandomErasing(p=0.1),

    v2.ToDtype(torch.float, scale=True),
    v2.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]),
])

transforms_test = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize((224, 224), antialias=True),
    v2.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]),
])


cutmix = v2.CutMix(num_classes=num_classes, alpha=1.0)
mixup = v2.MixUp(num_classes=num_classes, alpha=0.2)
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])

if CUT_UP_MIX:
    def collate_fn(batch):
        return cutmix_or_mixup(*default_collate(batch))
else:
    collate_fn = None

if HPC:
    dataset_path_prefix = "/apps/local/shared/CV703/datasets/"
else:
    dataset_path_prefix = "datasets/"

if dataset_name == 'CUB':
    dataset_path = dataset_path_prefix + "CUB/CUB_200_2011"

    train_simple_dataset = CUBDataset(image_root_path=dataset_path, transform=transforms_test, split="train")
    train_dataset = CUBDataset(image_root_path=dataset_path, transform=transforms_train, split="train")
    test_dataset = CUBDataset(image_root_path=dataset_path, transform=transforms_test, split="test")

    train_simple_loader = DataLoader(train_simple_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

    class_names = train_dataset.classes

elif dataset_name == 'CUB and FGVC-Aircraft':

    dataset_path_cub = dataset_path_prefix + "CUB/CUB_200_2011"
    train_simple_dataset_cub = CUBDataset(image_root_path=dataset_path_cub, transform=transforms_test, split="train")
    train_dataset_cub = CUBDataset(image_root_path=dataset_path_cub, transform=transforms_train, split="train")
    test_dataset_cub = CUBDataset(image_root_path=dataset_path_cub, transform=transforms_test, split="test")

    dataset_path_aircraft = dataset_path_prefix + "fgvc-aircraft-2013b"
    train_simple_dataset_aircraft = FGVCAircraft(root=dataset_path_aircraft, transform=transforms_test, train=True)
    train_dataset_aircraft = FGVCAircraft(root=dataset_path_aircraft, transform=transforms_train, train=True)
    test_dataset_aircraft = FGVCAircraft(root=dataset_path_aircraft, transform=transforms_test, train=False)

    concat_dataset_train_simple = ConcatDataset([train_simple_dataset_cub, train_simple_dataset_aircraft])
    concat_dataset_train = ConcatDataset([train_dataset_cub, train_dataset_aircraft])
    concat_dataset_test = ConcatDataset([test_dataset_cub, test_dataset_aircraft])

    train_simple_loader = torch.utils.data.DataLoader(
                concat_dataset_train_simple,
                batch_size=BATCH_SIZE, shuffle=True,
                num_workers=8, pin_memory=True,
                collate_fn=None
                )
    train_loader = torch.utils.data.DataLoader(
                concat_dataset_train,
                batch_size=BATCH_SIZE, shuffle=True,
                num_workers=8, pin_memory=True,
                collate_fn=collate_fn
                )
    test_loader = torch.utils.data.DataLoader(
                concat_dataset_test,
                batch_size=BATCH_SIZE, shuffle=False,
                num_workers=8, pin_memory=True
                )
    
    classes_1 = concat_dataset_train.datasets[0].classes
    classes_2 = concat_dataset_train.datasets[1].classes

    class_names = [*classes_1, *classes_2]

elif dataset_name == 'FoodX':
    dataset_path = dataset_path_prefix + "FoodX/food_dataset"

    train_dataset = FOODDataset(data_dir=dataset_path, transform=transforms_train, split="train")
    test_dataset = FOODDataset(data_dir=dataset_path, transform=transforms_test, split="val")

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)






In [5]:
state_dicts = []
val_results = []
test_results = []

#if there is no test directory create it
if not os.path.exists("test"):
    os.makedirs("test")

#if there is no folder named after dataset inside test directory create it
if not os.path.exists(os.path.join("test", dataset_name, MODEL)):
    os.makedirs(os.path.join("test", dataset_name, MODEL))

test_save_path = os.path.join("test", dataset_name, MODEL)

In [6]:
def souping(model, state_dicts, alphal):
    sd = {k : state_dicts[0][k].clone() * alphal[0] for k in state_dicts[0].keys()}
    for i in range(1, len(state_dicts)):
        for k in state_dicts[i].keys():
            sd[k] = sd[k] + state_dicts[i][k].clone() * alphal[i]
    model.load_state_dict(sd)
    return model

In [7]:
def greedy_souping(state_dicts, val_results, val_loader, loss, DEVICE, num_classes):
    ranked_candidates = [i for i in range(len(state_dicts))]
    ranked_candidates.sort(key=lambda x: -val_results[x])
    print(ranked_candidates)
    current_best = val_results[ranked_candidates[0]]
    print('currentttttttt bestttttttt', current_best)
    best_ingredients = ranked_candidates[:1]

    for i in range(1, len(state_dicts)):
        # add current index to the ingredients
        ingredient_indices = best_ingredients + [ranked_candidates[i]]
        alphal = [0 for i in range(len(state_dicts))]
        for j in ingredient_indices:
            alphal[j] = 1 / len(ingredient_indices)

        # benchmark and conditionally append
        model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

        greedy_model = souping(model, state_dicts, alphal)
        greedy_model.to(DEVICE)
        greedy_val_loss, greedy_val_acc = val_step(greedy_model, val_loader, loss, DEVICE)
        print(f'Models {ingredient_indices} got {greedy_val_acc} on validation.')
        if greedy_val_acc > current_best:
            current_best = greedy_val_acc
            best_ingredients = ingredient_indices
       

    alphal = [0 for i in range(len(state_dicts))]
    for j in best_ingredients:
        alphal[j] = 1 / len(best_ingredients)
    greedy_model = souping(model, state_dicts, alphal)
    return greedy_model, best_ingredients


In [8]:
def greedy_souping_2(state_dicts, val_results, val_loader, loss, DEVICE, num_classes):
    ranked_candidates = [i for i in range(len(state_dicts))]
    ranked_candidates.sort(key=lambda x: -val_results[x])
    print(ranked_candidates)
    current_best = val_results[ranked_candidates[0]]
    print('currentttttttt bestttttttt', current_best)
    best_ingredients = ranked_candidates[:1]
    should_restart = True
    while should_restart:
        should_restart = False
        included = []
        for i in range(0, len(state_dicts)):
            if ranked_candidates[i] in included:
                break
            if ranked_candidates[i] in best_ingredients:
                continue
            # add current index to the ingredients
            ingredient_indices = best_ingredients + [ranked_candidates[i]]
            included.append(ranked_candidates[i])
            alphal = [0 for i in range(len(state_dicts))]
            for j in ingredient_indices:
                alphal[j] = 1 / len(ingredient_indices)

            # benchmark and conditionally append
            model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

            greedy_model = souping(model, state_dicts, alphal)
            greedy_model.to(DEVICE)
            greedy_val_loss, greedy_val_acc = val_step(greedy_model, val_loader, loss, DEVICE)
            print(f'Models {ingredient_indices} got {greedy_val_acc} on validation.')
            if greedy_val_acc > current_best:
                should_restart = True
                included = []
                current_best = greedy_val_acc
                best_ingredients = ingredient_indices
       

    alphal = [0 for i in range(len(state_dicts))]
    for j in best_ingredients:
        alphal[j] = 1 / len(best_ingredients)
    greedy_model = souping(model, state_dicts, alphal)
    return greedy_model, best_ingredients


In [9]:
import random
                                        
def greedy_souping_3_random(state_dicts, val_results, val_loader, loss, DEVICE, num_classes, best_acc):
    ranked_candidates = [i for i in range(len(state_dicts))]
    # ranked_candidates.sort(key=lambda x: -val_results[x])
    print(ranked_candidates)
    current_best = best_acc
    print('currentttttttt bestttttttt', current_best)
    best_ingredients = ranked_candidates
    i_s = []
    while True:
        if len(best_ingredients) == 0:
            break
        
        b = [x for x in range(len(best_ingredients)) if x not in i_s]
        
        if len(b) == 0:
            break

        i = random.choice(b)
        i_s.append(i)
        
        ingredient_indices = best_ingredients[:i] + best_ingredients[i+1:]
        alphal = [0 for i in range(len(state_dicts))]
        for j in ingredient_indices:
            alphal[j] = 1 / len(ingredient_indices)
        model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

        greedy_model = souping(model, state_dicts, alphal)
        greedy_model.to(DEVICE)
        greedy_val_loss, greedy_val_acc = val_step(greedy_model, val_loader, loss, DEVICE)
        print(f'Models {ingredient_indices} got {greedy_val_acc} on validation.')
        if greedy_val_acc > current_best:
            current_best = greedy_val_acc
            best_ingredients = ingredient_indices
            i_s = []
            # benchmark and conditionally append
            
       

    alphal = [0 for i in range(len(state_dicts))]
    for j in best_ingredients:
        alphal[j] = 1 / len(best_ingredients)
    greedy_model = souping(model, state_dicts, alphal)
    return greedy_model, best_ingredients


In [10]:
                                 
def greedy_souping_3_sorted(state_dicts, val_results, val_loader, loss, DEVICE, num_classes, best_acc):
    ranked_candidates = [i for i in range(len(state_dicts))]
    ranked_candidates.sort(key=lambda x: val_results[x])
    print(ranked_candidates)
    current_best = best_acc
    print('currentttttttt bestttttttt', current_best)
    best_ingredients = ranked_candidates
    i = 0
    start = True
    while True:
        if len(best_ingredients) == 0:
            break

        if i >= len(best_ingredients):
            break
        if start:
            ingredient_indices = best_ingredients
            start = False
        else:
            ingredient_indices = best_ingredients[:i] + best_ingredients[i+1:]
        alphal = [0 for i in range(len(state_dicts))]
        for j in ingredient_indices:
            alphal[j] = 1 / len(ingredient_indices)
        model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

        greedy_model = souping(model, state_dicts, alphal)
        greedy_model.to(DEVICE)
        greedy_val_loss, greedy_val_acc = val_step(greedy_model, val_loader, loss, DEVICE)
        print(f'Models {ingredient_indices} got {greedy_val_acc} on validation.')
        i+=1
        if greedy_val_acc > current_best:
            current_best = greedy_val_acc
            best_ingredients = ingredient_indices
            i = 0
            # benchmark and conditionally append
        
       

    alphal = [0 for i in range(len(state_dicts))]
    for j in best_ingredients:
        alphal[j] = 1 / len(best_ingredients)
    greedy_model = souping(model, state_dicts, alphal)
    return greedy_model, best_ingredients

In [13]:
for idx, folder in enumerate(sorted(os.listdir(WEIGHT_PATH))):

    model_path = os.path.join(WEIGHT_PATH, folder)
    #read config in train_summary.json
    train_summary = json.load(open(os.path.join(model_path, "train_summary.json"), 'r'))
    model_config = train_summary["config"]
    
    #load model
    checkpoint = torch.load(os.path.join(model_path, "best_model.pth"), map_location=DEVICE)
    state_dicts.append(checkpoint)
    model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)
    model.load_state_dict(checkpoint)
    model = model.to(DEVICE)
    #load loss function
    LOSS = model_config["LOSS"]
    if LOSS == "MSE":
        loss = torch.nn.MSELoss()
    elif LOSS == "L1Loss":
        loss = torch.nn.L1Loss()
    elif LOSS == "SmoothL1Loss":
        loss = torch.nn.SmoothL1Loss()
    elif LOSS == "CrossEntropyLoss":
        loss = torch.nn.CrossEntropyLoss()
    elif LOSS == "BCEWithLogitsLoss":
        loss = torch.nn.BCEWithLogitsLoss()
    
    val_loss, val_acc = val_step(model, test_loader, loss, device = DEVICE)

    val_results.append({'Model Name': idx,
                                    'Val Accuracy': val_acc,
                                    'LR': train_summary["results"]["learning_rate"][0],
                                    })





/home/nuren.zhaksylyk/.conda/envs/CV701_env/lib/python3.8/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
100%|##########| 286/286 [00:43<00:00,  6.50it/s]


# ***ConvNeXtBase --- CUB***

In [12]:
results_val_df = pd.DataFrame(val_results)

val_copy = results_val_df.copy()
sorted_val = val_copy.sort_values(by= 'Val Accuracy',ascending=False)
sorted_val.to_csv(os.path.join(test_save_path, "VAL_RESULTS.csv"), index=False)

print(f"Best model Val Accuracy: {sorted_val.iloc[0]['Val Accuracy']}")
print(f"Second best model Val Accuracy: {sorted_val.iloc[1]['Val Accuracy']}")
print(f"Worst model Val Accuracy: {sorted_val.iloc[-1]['Val Accuracy']}")

#UNIFORM
print("Uniform souping ...")
alphal = [1 / len(state_dicts) for i in range(len(state_dicts))]
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)
uniform_model = souping(model, state_dicts, alphal)
uniform_model.to(DEVICE)

uniform_test_loss, uniform_test_acc  = val_step(uniform_model, test_loader, loss, DEVICE)
print(f"Uniform souping accuracy: {uniform_test_acc}")

#greedy

print("Greedy souping ...")
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

val_res = list(results_val_df['Val Accuracy'])
greedy_model, best_ingredients = greedy_souping(state_dicts, val_res, test_loader, loss, DEVICE, num_classes)
greedy_model.to(DEVICE)
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc = val_step(greedy_model, test_loader, loss, DEVICE)

print(f"Greedy souping accuracy: {greedy_test_acc}")


print("Greedy souping version 2 ...")
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

val_res = list(results_val_df['Val Accuracy'])
greedy_model, best_ingredients = greedy_souping_2(state_dicts, val_res, test_loader, loss, DEVICE, num_classes)
greedy_model.to(DEVICE)
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc = val_step(greedy_model, test_loader, loss, DEVICE)

print(f"Greedy souping accuracy: {greedy_test_acc}")

print("Greedy souping version 3 ...")
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

val_res = list(results_val_df['Val Accuracy'])
greedy_model, best_ingredients = greedy_souping_3_sorted(state_dicts, val_res, test_loader, loss, DEVICE, num_classes, sorted_val.iloc[0]['Val Accuracy'])
greedy_model.to(DEVICE)
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc = val_step(greedy_model, test_loader, loss, DEVICE)

print(f"Greedy souping accuracy: {greedy_test_acc}")



Best model Val Accuracy: 0.8797031411805316
Second best model Val Accuracy: 0.8797031411805316
Worst model Val Accuracy: 0.5918191232309286
Uniform souping ...


100%|##########| 182/182 [00:28<00:00,  6.47it/s]


Uniform souping accuracy: 0.880738695201933
Greedy souping ...
[6, 9, 5, 4, 11, 8, 10, 7, 0, 1, 2, 3]
currentttttttt bestttttttt 0.8797031411805316


100%|##########| 182/182 [00:28<00:00,  6.48it/s]


Models [6, 9] got 0.881429064549534 on validation.


...

Models [1, 7, 11, 4, 5, 9] got 0.8826372109078356 on validation.


100%|##########| 182/182 [00:28<00:00,  6.43it/s]


Models [1, 7, 11, 4, 5, 6] got 0.8828098032447359 on validation.
VAL INGREDIENTS [1, 7, 11, 4, 5, 6, 9]


100%|##########| 182/182 [00:28<00:00,  6.44it/s]

Greedy souping accuracy: 0.8828098032447359


# ***ConvNeXtTiny --- CUB***

In [15]:
results_val_df = pd.DataFrame(val_results)

val_copy = results_val_df.copy()
sorted_val = val_copy.sort_values(by= 'Val Accuracy',ascending=False)
sorted_val.to_csv(os.path.join(test_save_path, "VAL_RESULTS.csv"), index=False)

print(f"Best model Val Accuracy: {sorted_val.iloc[0]['Val Accuracy']}")
print(f"Second best model Val Accuracy: {sorted_val.iloc[1]['Val Accuracy']}")
print(f"Worst model Val Accuracy: {sorted_val.iloc[-1]['Val Accuracy']}")

#UNIFORM
print("Uniform souping ...")
alphal = [1 / len(state_dicts) for i in range(len(state_dicts))]
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)
uniform_model = souping(model, state_dicts, alphal)
uniform_model.to(DEVICE)

uniform_test_loss, uniform_test_acc  = val_step(uniform_model, test_loader, loss, DEVICE)
print(f"Uniform souping accuracy: {uniform_test_acc}")

#greedy

print("Greedy souping ...")
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

val_res = list(results_val_df['Val Accuracy'])
greedy_model, best_ingredients = greedy_souping(state_dicts, val_res, test_loader, loss, DEVICE, num_classes)
greedy_model.to(DEVICE)
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc = val_step(greedy_model, test_loader, loss, DEVICE)

print(f"Greedy souping accuracy: {greedy_test_acc}")



print("Greedy souping version 3 ...")
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

val_res = list(results_val_df['Val Accuracy'])
greedy_model, best_ingredients = greedy_souping_3_sorted(state_dicts, val_res, test_loader, loss, DEVICE, num_classes, sorted_val.iloc[0]['Val Accuracy'])
greedy_model.to(DEVICE)
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc = val_step(greedy_model, test_loader, loss, DEVICE)

print(f"Greedy souping accuracy: {greedy_test_acc}")




Best model Val Accuracy: 0.8626164998274076
Second best model Val Accuracy: 0.8603727994477045
Worst model Val Accuracy: 0.8117017604418364
Uniform souping ...


100%|##########| 182/182 [00:17<00:00, 10.23it/s]


Uniform souping accuracy: 0.8629616845012081
Greedy souping ...
[4, 6, 0, 5, 10, 11, 9, 8, 7, 1, 2, 3]
currentttttttt bestttttttt 0.8626164998274076


100%|##########| 182/182 [00:16<00:00, 11.04it/s]


Models [4, 6] got 0.8631342768381084 on validation.


...

Models [1, 5, 10, 0, 4] got 0.8658957542285123 on validation.


100%|##########| 182/182 [00:18<00:00, 10.11it/s]


Models [1, 5, 10, 0, 6] got 0.865723161891612 on validation.
VAL INGREDIENTS [1, 5, 10, 0, 6, 4]


100%|##########| 182/182 [00:15<00:00, 12.03it/s]

Greedy souping accuracy: 0.8669313082499137


In [18]:
print("Greedy souping version 2 ...")
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

val_res = list(results_val_df['Val Accuracy'])
greedy_model, best_ingredients = greedy_souping_2(state_dicts, val_res, test_loader, loss, DEVICE, num_classes)
greedy_model.to(DEVICE)
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc = val_step(greedy_model, test_loader, loss, DEVICE)

print(f"Greedy souping accuracy: {greedy_test_acc}")

Greedy souping version 2 ...


/home/nuren.zhaksylyk/.conda/envs/CV701_env/lib/python3.8/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


[4, 6, 0, 5, 10, 11, 9, 8, 7, 1, 2, 3]
currentttttttt bestttttttt 0.8626164998274076


100%|##########| 182/182 [00:15<00:00, 11.85it/s]


Models [4, 6] got 0.8631342768381084 on validation.


...

Models [4, 6, 10, 1, 11, 0, 2] got 0.8653779772178115 on validation.


100%|##########| 182/182 [00:15<00:00, 11.85it/s]


Models [4, 6, 10, 1, 11, 0, 3] got 0.8636520538488092 on validation.
VAL INGREDIENTS [4, 6, 10, 1, 11, 0]


100%|##########| 182/182 [00:15<00:00, 11.80it/s]


Greedy souping accuracy: 0.8669313082499137


# ***ConvNeXtBase --- CUB and FGVC-Aircraft***

In [14]:
results_val_df = pd.DataFrame(val_results)

val_copy = results_val_df.copy()
sorted_val = val_copy.sort_values(by= 'Val Accuracy',ascending=False)
sorted_val.to_csv(os.path.join(test_save_path, "VAL_RESULTS.csv"), index=False)

print(f"Best model Val Accuracy: {sorted_val.iloc[0]['Val Accuracy']}")
print(f"Second best model Val Accuracy: {sorted_val.iloc[1]['Val Accuracy']}")
print(f"Worst model Val Accuracy: {sorted_val.iloc[-1]['Val Accuracy']}")

#UNIFORM
print("Uniform souping ...")
alphal = [1 / len(state_dicts) for i in range(len(state_dicts))]
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)
uniform_model = souping(model, state_dicts, alphal)
uniform_model.to(DEVICE)

uniform_test_loss, uniform_test_acc  = val_step(uniform_model, test_loader, loss, DEVICE)
print(f"Uniform souping accuracy: {uniform_test_acc}")

#greedy

print("Greedy souping ...")
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

val_res = list(results_val_df['Val Accuracy'])
greedy_model, best_ingredients = greedy_souping(state_dicts, val_res, test_loader, loss, DEVICE, num_classes)
greedy_model.to(DEVICE)
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc = val_step(greedy_model, test_loader, loss, DEVICE)

print(f"Greedy souping accuracy: {greedy_test_acc}")


print("Greedy souping version 2 ...")
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

val_res = list(results_val_df['Val Accuracy'])
greedy_model, best_ingredients = greedy_souping_2(state_dicts, val_res, test_loader, loss, DEVICE, num_classes)
greedy_model.to(DEVICE)
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc = val_step(greedy_model, test_loader, loss, DEVICE)

print(f"Greedy souping accuracy: {greedy_test_acc}")


print("Greedy souping version 3 ...")
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

val_res = list(results_val_df['Val Accuracy'])
greedy_model, best_ingredients = greedy_souping_3_sorted(state_dicts, val_res, test_loader, loss, DEVICE, num_classes, sorted_val.iloc[0]['Val Accuracy'])
greedy_model.to(DEVICE)
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc = val_step(greedy_model, test_loader, loss, DEVICE)

print(f"Greedy souping accuracy: {greedy_test_acc}")



Best model Val Accuracy: 0.8879149775391695
Second best model Val Accuracy: 0.8869288922975787
Worst model Val Accuracy: 0.5117782403856689
Uniform souping ...


100%|##########| 286/286 [00:42<00:00,  6.74it/s]


Uniform souping accuracy: 0.8866001972170483
Greedy souping ...
[5, 4, 6, 7, 9, 8, 0, 11, 10, 1, 2, 3]
currentttttttt bestttttttt 0.8879149775391695


100%|##########| 286/286 [00:43<00:00,  6.52it/s]


Models [5, 4] got 0.8871480223512654 on validation.


...

Models [3, 2, 1, 10, 11, 0, 8, 9, 7, 6, 5] got 0.8869288922975787 on validation.


100%|##########| 286/286 [00:44<00:00,  6.48it/s]


Models [3, 2, 1, 10, 11, 0, 8, 9, 7, 6, 4] got 0.8868193272707352 on validation.
VAL INGREDIENTS [3, 2, 1, 10, 11, 0, 8, 9, 7, 6, 4, 5]


100%|##########| 286/286 [00:47<00:00,  5.96it/s]

Greedy souping accuracy: 0.8866001972170483


# ***ConvNeXtTiny --- CUB and FGVC-Aircraft***

In [12]:
results_val_df = pd.DataFrame(val_results)

val_copy = results_val_df.copy()
sorted_val = val_copy.sort_values(by= 'Val Accuracy',ascending=False)
sorted_val.to_csv(os.path.join(test_save_path, "VAL_RESULTS.csv"), index=False)

print(f"Best model Val Accuracy: {sorted_val.iloc[0]['Val Accuracy']}")
print(f"Second best model Val Accuracy: {sorted_val.iloc[1]['Val Accuracy']}")
print(f"Worst model Val Accuracy: {sorted_val.iloc[-1]['Val Accuracy']}")

#UNIFORM
print("Uniform souping ...")
alphal = [1 / len(state_dicts) for i in range(len(state_dicts))]
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)
uniform_model = souping(model, state_dicts, alphal)
uniform_model.to(DEVICE)

uniform_test_loss, uniform_test_acc  = val_step(uniform_model, test_loader, loss, DEVICE)
print(f"Uniform souping accuracy: {uniform_test_acc}")

#greedy

print("Greedy souping ...")
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

val_res = list(results_val_df['Val Accuracy'])
greedy_model, best_ingredients = greedy_souping(state_dicts, val_res, test_loader, loss, DEVICE, num_classes)
greedy_model.to(DEVICE)
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc = val_step(greedy_model, test_loader, loss, DEVICE)

print(f"Greedy souping accuracy: {greedy_test_acc}")


print("Greedy souping version 2 ...")
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

val_res = list(results_val_df['Val Accuracy'])
greedy_model, best_ingredients = greedy_souping_2(state_dicts, val_res, test_loader, loss, DEVICE, num_classes)
greedy_model.to(DEVICE)
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc = val_step(greedy_model, test_loader, loss, DEVICE)

print(f"Greedy souping accuracy: {greedy_test_acc}")


print("Greedy souping version 3 ...")
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

val_res = list(results_val_df['Val Accuracy'])
greedy_model, best_ingredients = greedy_souping_3_sorted(state_dicts, val_res, test_loader, loss, DEVICE, num_classes, sorted_val.iloc[0]['Val Accuracy'])
greedy_model.to(DEVICE)
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc = val_step(greedy_model, test_loader, loss, DEVICE)

print(f"Greedy souping accuracy: {greedy_test_acc}")



Best model Val Accuracy: 0.8690697929220993
Second best model Val Accuracy: 0.8686315328147255
Worst model Val Accuracy: 0.5707242248274351
Uniform souping ...


100%|##########| 286/286 [00:44<00:00,  6.44it/s]


Uniform souping accuracy: 0.87356195902268
Greedy souping ...
[9, 7, 4, 5, 11, 8, 10, 0, 6, 1, 2, 3]
currentttttttt bestttttttt 0.8690697929220993


100%|##########| 286/286 [00:44<00:00,  6.45it/s]


Models [9, 7] got 0.8710419634052811 on validation.


...

Models [2, 1, 6, 11, 5, 4, 9] got 0.8741097841568971 on validation.


100%|##########| 286/286 [00:47<00:00,  6.07it/s]


Models [2, 1, 6, 11, 5, 4, 7] got 0.8748767393448011 on validation.
VAL INGREDIENTS [2, 1, 6, 11, 5, 4, 7, 9]


100%|##########| 286/286 [00:44<00:00,  6.41it/s]

Greedy souping accuracy: 0.875095869398488
